In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Utils import constants as Co
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as puti
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( puti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
print( Co.Omega())

In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}


In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('../YAML/configure_plotting.yaml', 'r') as file:
    cfgPlot = yaml.safe_load(file)


######################################################
season =  'ann' #cfgPlot['zonal_mean']['season']
valData  = cfgPlot['zonal_mean']['validation']['data']
valYears = cfgPlot['zonal_mean']['validation']['years']
print( f"Season= {season}" )
print( valData , valYears )
fld='V' #'U'


In [ ]:


exp_file = 'Utility.yaml'
xl= ['control','tausrf150','evap075'] 



exp_file=f'../YAML/{exp_file}'
with open( exp_file, 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                        hsPat='cam.h1a' , ymdPat=ymdPat, cmip_fld=fld,
                        verbose=True, open_dataset=True )
    A['Src']=Src
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    if 'desc' in cfg[x]:
        A['desc'] = cfg[x]['desc']
    Al.append( A )
    print( f'Vertcal levs - {A.X.lev.shape}' )
    print( f'{A.exp} Years {len(A.X.time) /12. } \n')


In [ ]:
########################################
# Calculate angluar momentum and regrid
# to lat-lon if needed
########################################
for A in Al:
    Hkey = A['Hkey']
    
    if (Hkey == 'yx' ):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        lon2d, lat2d = np.meshgrid(lon_a, lat_a)
        Me_yx = Co.Omega() * ( ( Co.Rearth() * np.cos(  lat2d * Co.pi() / 180. ) )**2 ) 
    
    elif (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        u_tzc = A.X.U.values 
        v_tzc = A.X.V.values 
        vu_tzc = A.X.VU.values 
        pint_tzc = A.X.PINT.values
        lat_c = A.X.lat.values 
        nt,nz,ncol = np.shape( u_tzc )
        Me_c = Co.Omega() * ( ( Co.Rearth() * np.cos(  lat_c * Co.pi() / 180. ) )**2 ) 
        Ma_tzc = np.zeros( (nt,nz,ncol) )
        Me_tzc = np.zeros( (nt,nz,ncol) )
        Mu_tzc = np.zeros( (nt,nz,ncol) )
        fMa_tzc = np.zeros( (nt,nz,ncol) )
        fMe_tzc = np.zeros( (nt,nz,ncol) )
        fMu_tzc = np.zeros( (nt,nz,ncol) )
        print(' Got data ' )
        #delp_tzc = np.zeros( (nt,nz,ncol) )
        for t in np.arange( nt ):
            for z in np.arange( nz ):
                Mu_c = u_tzc[t,z,:] *( ( Co.Rearth() * np.cos(  lat_c * Co.pi() / 180. ) ) )
                Ma_tzc[t,z,:] = Mu_c + Me_c
                Mu_tzc[t,z,:] = Mu_c
                Me_tzc[t,z,:] = Me_c
                fMu_tzc[t,z,:] = vu_tzc[t,z,:] *( ( Co.Rearth() * np.cos(  lat_c * Co.pi() / 180. ) ) )
                fMe_tzc[t,z,:] = v_tzc[t,z,:] * Me_c

        delp_tzc = pint_tzc[:, 1:, :] - pint_tzc[:, :-1, :]
        print(' Made AAM ... ' )

        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
        dlon = ( lon_a[1]-lon_a[0] ) * Co.pi() / 180.
        Lon_a, Lat_a = np.meshgrid(lon_a, lat_a)

        nz,ny,nx = len( lev_a ) , len( lat_a ) , len( lon_a )
        Lat3D = np.tile(Lat_a, (nz, 1, 1)).reshape( nz,ny,nx )
        aCos3D = Co.Rearth() * np.cos( Lat3D * Co.pi() / 180. ) 

        A['zlev_a']= zlev_a
        A['lat_a'] = lat_a
        A['lon_a'] = lon_a
        
        print(' Made grid and scaling ... ' )

        
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        Ma_tzyx   = RgF.Horz(xfld_Src=Ma_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        Me_tzyx   = RgF.Horz(xfld_Src=Me_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        Mu_tzyx   = RgF.Horz(xfld_Src=Mu_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        fMe_tzyx  = RgF.Horz(xfld_Src=fMe_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        fMu_tzyx  = RgF.Horz(xfld_Src=fMu_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        delp_tzyx = RgF.Horz(xfld_Src=delp_tzc , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

        A['Me_tzyx']  = Me_tzyx
        A['Mu_tzyx']  = Mu_tzyx
        A['fMe_tzyx'] = fMe_tzyx
        A['fMu_tzyx'] = fMu_tzyx

        fMa_tzyx = fMu_tzyx + fMe_tzyx
        dpfMa_tyzx = delp_tzyx * fMa_tzyx / Co.grav()
        dpfMa_tyzx = dpfMa_tyzx *aCos3D

        dpfMa_tyzx_Iz = np.sum( dpfMa_tyzx , axis=1 )
        

        dpfMa_tyzx_Izx=np.average( dpfMa_tyzx_Iz , axis=2 ) 

        dpfMa_tyzx_Izxt=np.average( dpfMa_tyzx_Izx , axis=0 )

        A['dpfMa_tyzx_Iz']   =  dpfMa_tyzx_Iz
        A['dpfMa_tyzx_Izx']  =  dpfMa_tyzx_Izx
        A['dpfMa_tyzx_Izxt'] =  dpfMa_tyzx_Izxt


In [ ]:
#plt.plot( dpfMa_tyzx_Izxt * np.cos(  lat_a * Co.pi() / 180. )  )
for A in Al:
    plt.plot( lat_a, A['dpfMa_tyzx_Izxt' ] , label=A['exp'] ) #* np.cos(  lat_a * Co.pi() / 180. )  ) 
    #plt.plot( A['dpfMa_tyzx_Izxt' ]  * np.cos(  lat_a * Co.pi() / 180. )  ) 
    ylim=np.array([-1.,0]) *1.e19
    plt.ylim(ylim)

plt.legend()

In [ ]:
Al[0]['exp']

In [ ]:
A0 = np.average(np.average( Al[0]['fMu_tzyx'] ,axis=3 ), axis=0 ) 
A1 = np.average(np.average( Al[1]['fMu_tzyx'] ,axis=3 ), axis=0 ) 
A2 = np.average(np.average( Al[2]['fMu_tzyx'] ,axis=3 ), axis=0 ) 


In [ ]:
lev=0.1*np.linspace(-1.5,1,num=21)*1.e9
plt.contour( lat_a, zlev_a, (A0.T *lev_a/1_000.).T , colors='blue', levels=lev )
plt.contour( lat_a, zlev_a, (A1.T *lev_a/1_000.).T, colors='orange', levels=lev )
plt.contour( lat_a, zlev_a, (A2.T *lev_a/1_000.).T, colors='green', levels=lev )
plt.ylim(0,5)
plt.colorbar()

In [ ]:
y= np.argmin( np.abs(lat_a+35))
plt.plot( A0[:,y]*lev_a/1_000.,zlev_a )
plt.plot( A1[:,y]*lev_a/1_000.,zlev_a )
plt.plot( A2[:,y]*lev_a/1_000.,zlev_a )
plt.ylim(0,15)


In [ ]:
t=0
lev=np.linspace(-1e9,1e9,num=21)
fig,ax=plt.subplots( 1,3, figsize=(20,7))
#plt.contourf( lat_a, zlev_a, np.average( np.average( Mu_tzyx[:,:,:,:], axis=0) , axis=2 ) / ( ( Co.Rearth() * np.cos(  lat_a * Co.pi() / 180. ) ) ) , levels=21 )
ax[0].contourf( lat_a, zlev_a, np.average( np.average( (1*fMu_tzyx+1*fMe_tzyx), axis=0) , axis=2 )  , levels=lev )
ax[1].contourf( lat_a, zlev_a, np.average( np.average( (0*fMu_tzyx+1*fMe_tzyx), axis=0) , axis=2 )  , levels=lev )
ax[2].contourf( lat_a, zlev_a, np.average( np.average( (1*fMu_tzyx+0*fMe_tzyx), axis=0) , axis=2 )  , levels=lev )
#plt.colorbar()

In [ ]:
fMa_tzyx = fMu_tzyx + fMe_tzyx
dpfMa_tyzx = delp_tzyx * fMa_tzyx

In [ ]:
dpfMa_tyzx_Iz = np.sum( dpfMa_tyzx , axis=1 )
dpfMa_tyzx_Iz.shape

In [ ]:
dpfMa_tyzx_Izx=np.average( dpfMa_tyzx_Iz , axis=2 ) # cosine (lat) weighting needed ???
dpfMa_tyzx_Izxt=np.average( dpfMa_tyzx_Izx , axis=0 )


In [ ]:
Lon_a, Lat_a = np.meshgrid(lon_a, lat_a)
poo=dpfMa_tyzx_Iz * np.cos(  Lat_a * Co.pi() / 180. )  
poo.shape
poo_Izx=np.average(poo , axis=2 )
poo_Izxt=np.average(poo_Izx , axis=0 )



In [ ]:
plt.plot( pint_tzc[0,:,20_000], 'o-' )
plt.plot( delp_tzc[0,:,20_000] )

In [ ]:
        
        # Zonal Av
        AAa_sz=np.average( AAa_s_xy , axis=2 )

    A['AA_sz'] = AAa_sz
    A['lonx'] = lon_a
    A['latx'] = lat_a
    A['zlev'] = zlev_a
    A['plev'] = lev_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
########################################
# Make seasonal means
########################################

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')
    
    AAa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld=fld,return_time=True)
    A['AA_s']=AAa_s
    A['years']=years_a
    A['months']=months_a


In [ ]:
####
# Validation
####
regrid_val_x_model = False #True

if (len(lev_a)==58):
    DstVgrid='L58'
if (len(lev_a)==93):
    DstVgrid='L93'

valYears='*'
Aval = Val.data(fld=fld,season=season,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
if (Aval['data_exists']==True):
    print( f'Found validation data for {fld}' )
    AAv_s , zlev_v,lat_v,lon_v, years_Uv, Src, SrcVgrid = Aval['aa'], Aval['lev'], Aval['lat'], Aval['lon'], Aval['years'], Aval['data_Hgrid'], Aval['data_Vgrid']
    
    if (regrid_val_x_model == True):
        PSval = Val.data(fld='PS',season=season,Years=valYears ) 
        RegridObj1 = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        RegridObj2 = GrU.regrid_object_lib(RgOb=RgObs, src='ERA5', dst=Dst)
        AAv_s_x1 =RgF.Horz(xfld_Src=AAv_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj1 )
        PSval_x1 =RgF.Horz(xfld_Src=PSval['aa'] , Src='ERA5', Dst=Dst , RegridObj_In= RegridObj2 )
        AAv_s,pmid_Dst,pmid_Src =RgF.Vert(DstVgrid=DstVgrid, DstTZHkey='zyx', SrcVgrid=SrcVgrid, xfld_Src=AAv_s_x1, ps_Src=PSval_x1, pmid_output=True )
        lon_v = Al[0]['lonx']
        lat_v = Al[0]['latx']
        zlev_v = Al[0]['zlev']
    
    AAv_sz=np.average( AAv_s , axis=2 )
else:
    print( f'No validation data for {fld}' )



In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
plot_bias_vs_obs = False #True
if (plot_bias_vs_obs == True):
    nxplo,nyplo=numx+2,1
else:
    nxplo,nyplo=numx+1,1


fig = plt.figure(figsize=(8*nxplo, 9))
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

ulev=np.linspace( -60,140,num=21) 
ulevx = np.append(ulev, [160,180,200])

ulev=np.linspace( -0.05,0.05,num=41) 
ulevx=ulev


####################################
# Use a centarlized functiin to define 
# intervals 
####################################
#def contour_intervals( fld, **kwargs ): return flev,dlev,scale

#dlev=np.linspace( -20,20,num=21) 

ulev,dlev,scale = puti.contour_intervals( fld, zonal_mean=1 )
#ulev=np.linspace(-450,450,num=51)
#ulev=np.linspace(-5,5,num=51)
ulevx=ulev
print(dlev)
cmap='bwr_r' #'gist_ncar'
cmap='gist_ncar'
titlesize=24 #18

plotTop=85 #40 # 15 #10 #85
BoxParm =['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
BoxParm =['use_gw_convect_dp','use_gw_movmtn_pbl','use_gw_front','use_gw_rdg_resid','use_gw_rdg_beta','use_gw_oro']
BoxParm =['use_gw_movmtn_pbl','use_gw_rdg_resid','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','effgw_rdg_resid',]
BoxParm =['movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','taubgnd','effgw_cm','effgw_rdg_beta','effgw_rdg_resid',]
#BoxParm =['use_gw_movmtn_pbl','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl',]  #'effgw_rdg_resid',]


titles=[ r'1$^\circ$ control','14km simulation',r'1$^\circ$ w/ additional GW source','yy',]
titles=[ r'1$^\circ$ control',r'1$^\circ$ w/out GW parameterization','yy',]
ix=0

for A in Al:
    
    AAa_sz = A['AA_sz'] 
    lon_a = A['lonx']
    lat_a = A['latx']
    zlev_a = A['zlev']
    years_a = A['years']
    if 'gw_parameters' in A:
        parameters=A['gw_parameters']
    else:
        parameters=None

    # Create 2D arrays
    Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
    
    # Adding the hatching for the sponge layer
    sponge_layer = np.zeros_like(AAa_sz)
    sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1

    if ('desc' in A):
        title = f"{A.desc} \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    else:
        title = f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    #title = titles[ix]

    n=ix+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_a,zlev_a,AAa_sz ,levels=ulev, cmap=cmap )
    co2=ax1.contour(lat_a,zlev_a,AAa_sz ,levels=ulevx, colors='black')
    #ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    ax1.set_xlabel('Latitude', fontsize=16 )
    ax1.set_ylabel('Z (km)', fontsize=16 )
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)
    
    ix=ix+1

if (Aval['data_exists']==True):
    for ix in np.arange( start=numx,stop=numx+1 ):
    
        title = f"<{valData}> {season.upper()} {years_Uv} " 
        n=ix+1
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,AAv_sz ,levels=ulev, cmap=cmap )
        co2=ax1.contour(lat_v,zlev_v,AAv_sz ,levels=ulev, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        ax1.set_xlabel('Latitude', fontsize=16 )
        ax1.set_ylabel('Z (km)', fontsize=16 )
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')

if (Aval['data_exists']==True) and (regrid_val_x_model == True) and (plot_bias_vs_obs==True) :
    for ix in np.arange( start=numx+1,stop=numx+2 ):
    
        title = f"Bias vs <{valData}> {season.upper()} {years_Uv} " 
        n=ix+1
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,AAa_sz-AAv_sz ,levels=dlev/2, cmap='bwr' )
        co2=ax1.contour(lat_v,zlev_v,AAa_sz-AAv_sz ,levels=dlev/2, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        ax1.set_xlabel('Latitude', fontsize=16 )
        ax1.set_ylabel('Z (km)', fontsize=16 )
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')
  

FigName = f"AGU_2024_Cecile_poster_zonalUT.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"multiFig.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName )



In [ ]:
f= '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001/atm/proc/tseries/month_1/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.h0.U.195001-201412.nc'
poop=xr.open_dataset( f )


In [ ]:
poop.lev

In [ ]:
print(dlev)

In [ ]:
print( Al[2].zlev)

In [ ]:

print( Al[2]['zlev'][-1] * 1000. )
print( Al[2]['zlev'][-1-5] * 1000. )
print( Al[2]['zlev'][-1-10] * 1000. )
print( Al[2]['zlev'][-1-20] * 1000. )
print( Al[2]['zlev'][-1-30] * 1000. )


In [ ]:

print( Al[2]['plev'][-1] * 100. )
print( Al[2]['plev'][-1-20] * 100. )
print( Al[2]['plev'][-1-30] * 100. )

print( Al[2]['plev'][-1-1] * 100. )
print( Al[2]['plev'][-1-1-10] * 100. )
print( Al[2]['plev'][-1-5] * 100. )
print( Al[2]['plev'][-1-5-10] * 100. )


In [ ]:
Al[0].X

In [ ]:
qrs=Al[2].X.QRS.values

In [ ]:
plt.plot( qrs[70,10,:] )

In [ ]:
print( Al[0].zlev[8] )
print( Al[0].zlev[12] )
print( Al[0].zlev[16] )

In [ ]:

plt.plot( Al[0].UU_sz[8,:] )
plt.plot( Al[1].UU_sz[8,:] )
plt.plot( Al[2].UU_sz[8,:] )
#plt.plot( Al[0].UU_sz[8,:] )
